Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of Cybertron package.

The Cybertron is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

Cybertron tutorial 03: Use normalized dataset and validation dataset

In [1]:
import sys
import time
import numpy as np
from mindspore import nn
from mindspore import context
from mindspore import dataset as ds
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig
from cybertron import Cybertron
from cybertron import MolCT
from cybertron import AtomwiseReadout
from cybertron.train import TrainMonitor, MAE, MLoss
from cybertron.train import WithLabelLossCell, WithLabelEvalCell

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [2]:
data_name = sys.path[0] + '/dataset_qm9_normed_'
train_file = data_name + 'trainset_1024.npz'
valid_file = data_name + 'validset_128.npz'

train_data = np.load(train_file)
valid_data = np.load(valid_file)

idx = [7]  # U0

num_atom = int(train_data['num_atoms'])
scale = train_data['scale'][idx]
shift = train_data['shift'][idx]
ref = train_data['type_ref'][:, idx]

In [3]:
mod = MolCT(
    cutoff=1,
    n_interaction=3,
    dim_feature=128,
    n_heads=8,
    activation='swish',
    max_cycles=1,
    length_unit='nm',
)

In [4]:
readout = AtomwiseReadout(mod, dim_output=1)

In [5]:
net = Cybertron(model=mod, readout=readout, dim_output=1,
                num_atoms=num_atom, length_unit='nm')
net.print_info()

Cybertron Engine, Ride-on!
--------------------------------------------------------------------------------
    Length unit: nm
    Input unit scale: 1
--------------------------------------------------------------------------------
    Deep molecular model:  MolCT
--------------------------------------------------------------------------------
       Length unit: nm
       Atom embedding size: 64
       Cutoff distance: 1.0 nm
       Radical basis function (RBF): LogGaussianBasis
          Minimum distance: 0.04 nm
          Maximum distance: 1.0 nm
          Reference distance: 1.0 nm
          Log Gaussian begin: -3.218876
          Log Gaussian end: 0.006724119
          Interval for log Gaussian: 0.0512
          Sigma for log gaussian: 0.3
          Number of basis functions: 64
          Rescale the range of RBF to (-1,1).
       Calculate distance: Yes
       Calculate bond: No
       Feature dimension: 128
-----------------------------------------------------------------------

In [6]:
tot_params = 0
for i, param in enumerate(net.get_parameters()):
    tot_params += param.size
    print(i, param.name, param.shape)
print('Total parameters: ', tot_params)

0 model.atom_embedding.embedding_table (64, 128)
1 model.dis_filter.linear.weight (128, 64)
2 model.dis_filter.linear.bias (128,)
3 model.dis_filter.residual.nonlinear.mlp.0.weight (128, 128)
4 model.dis_filter.residual.nonlinear.mlp.0.bias (128,)
5 model.dis_filter.residual.nonlinear.mlp.1.weight (128, 128)
6 model.dis_filter.residual.nonlinear.mlp.1.bias (128,)
7 model.interactions.0.positional_embedding.norm.gamma (128,)
8 model.interactions.0.positional_embedding.norm.beta (128,)
9 model.interactions.0.positional_embedding.x2q.weight (128, 128)
10 model.interactions.0.positional_embedding.x2k.weight (128, 128)
11 model.interactions.0.positional_embedding.x2v.weight (128, 128)
12 model.interactions.0.multi_head_attention.output.weight (128, 128)
13 model.interactions.1.positional_embedding.norm.gamma (128,)
14 model.interactions.1.positional_embedding.norm.beta (128,)
15 model.interactions.1.positional_embedding.x2q.weight (128, 128)
16 model.interactions.1.positional_embedding.x2k.

In [7]:
N_EPOCH = 8
REPEAT_TIME = 1
BATCH_SIZE = 32

In [8]:
ds_train = ds.NumpySlicesDataset(
    {'R': train_data['R'], 'Z': train_data['Z'], 'E': train_data['E'][:, idx]}, shuffle=True)
ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)
ds_train = ds_train.repeat(REPEAT_TIME)

In [9]:
ds_valid = ds.NumpySlicesDataset(
    {'R': valid_data['R'], 'Z': valid_data['Z'], 'E': valid_data['E'][:, idx]}, shuffle=False)
ds_valid = ds_valid.batch(128)
ds_valid = ds_valid.repeat(1)
eval_network = WithLabelEvalCell(
    'RZE', net, nn.MAELoss(), scale=scale, shift=shift, type_ref=ref)

WithLabelEvalCell with input type: RZE
   with scaleshift for training and evaluate dataset:
   Scale: [18.248537]
   Shift: [-409.42038]
   Scaleshift mode: atomwise
   with reference value for atom types:
   Type     Label0
   0:        0.00e+00
   1:       -1.31e+03
   2:        0.00e+00
   3:        0.00e+00
   4:        0.00e+00
   5:        0.00e+00
   6:       -9.94e+04
   7:       -1.43e+05
   8:       -1.97e+05
   9:       -2.62e+05


In [10]:
loss_network = WithLabelLossCell('RZE', net, nn.MAELoss())
eval_network = WithLabelEvalCell('RZE', net, nn.MAELoss(), scale=scale, shift=shift, type_ref=ref)

WithLabelLossCell with input type: RZE
WithLabelEvalCell with input type: RZE
   with scaleshift for training and evaluate dataset:
   Scale: [18.248537]
   Shift: [-409.42038]
   Scaleshift mode: atomwise
   with reference value for atom types:
   Type     Label0
   0:        0.00e+00
   1:       -1.31e+03
   2:        0.00e+00
   3:        0.00e+00
   4:        0.00e+00
   5:        0.00e+00
   6:       -9.94e+04
   7:       -1.43e+05
   8:       -1.97e+05
   9:       -2.62e+05


In [11]:
lr = nn.ExponentialDecayLR(
    learning_rate=1e-3, decay_rate=0.96, decay_steps=4, is_stair=True)
optim = nn.Adam(params=net.trainable_params(), learning_rate=lr)

In [12]:
eval_mae = 'EvalMAE'
atom_mae = 'AtomMAE'
eval_loss = 'Evalloss'
model = Model(loss_network, optimizer=optim, eval_network=eval_network, metrics={
    eval_mae: MAE([1, 2]), atom_mae: MAE([1, 2, 3], averaged_by_atoms=True), eval_loss: MLoss(0)})

In [13]:
outdir = 'Tutorial_C03'
outname = outdir + '_' + net.model_name
record_cb = TrainMonitor(model, outname, per_step=16, avg_steps=16,
                         directory=outdir, eval_dataset=ds_valid, best_ckpt_metrics=eval_loss)

In [14]:
config_ck = CheckpointConfig(
    save_checkpoint_steps=32, keep_checkpoint_max=64, append_info=[net.hyper_param])
ckpoint_cb = ModelCheckpoint(
    prefix=outname, directory=outdir, config=config_ck)

In [15]:
print("Start training ...")
beg_time = time.time()
model.train(N_EPOCH, ds_train, callbacks=[record_cb, ckpoint_cb], dataset_sink_mode=False)
end_time = time.time()
used_time = end_time - beg_time
m, s = divmod(used_time, 60)
h, m = divmod(m, 60)
print("Training Fininshed!")
print("Training Time: %02d:%02d:%02d" % (h, m, s))

[WARNING] ME(888067:139901616760064,MainProcess):2023-03-29-10:47:27.295.658 [mindspore/train/model.py:1077] For TrainMonitor callback, {'epoch_end', 'begin', 'step_end'} methods may not be supported in later version, Use methods prefixed with 'on_train' or 'on_eval' instead when using customized callbacks.


Start training ...
Epoch: 1, Step: 16, Learning_rate: 0.0008847359, Last_Loss: 11.806259, Avg_loss: 19.56789082288742, EvalMAE: 205.1220703125, AtomMAE: 11.608316421508789, Evalloss: 11.240756034851074
Epoch: 1, Step: 32, Learning_rate: 0.00075144734, Last_Loss: 6.0831475, Avg_loss: 9.709901571273804, EvalMAE: 117.24072265625, AtomMAE: 6.892520904541016, Evalloss: 6.425112247467041
Epoch: 2, Step: 48, Learning_rate: 0.00063823926, Last_Loss: 4.529848, Avg_loss: 6.893972396850586, EvalMAE: 220.96826171875, AtomMAE: 12.511213302612305, Evalloss: 12.10911750793457
Epoch: 2, Step: 64, Learning_rate: 0.0005420862, Last_Loss: 3.6108932, Avg_loss: 7.804106295108795, EvalMAE: 95.375, AtomMAE: 5.5689697265625, Evalloss: 5.2261962890625
Epoch: 3, Step: 80, Learning_rate: 0.00046041902, Last_Loss: 4.8873215, Avg_loss: 5.610067039728165, EvalMAE: 76.38671875, AtomMAE: 4.544213771820068, Evalloss: 4.185667991638184
Epoch: 3, Step: 96, Learning_rate: 0.00039105528, Last_Loss: 5.439431, Avg_loss: 4.3